In [1]:
import pickle
import numpy as np
!pip install tensorflow==2.11 keras==2.11
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


2023-05-14 22:54:45.766885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 22:54:46.997885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-14 22:54:46.997940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-14 22:54:46.997944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

In [4]:
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Load the CIFAR-100 dataset
data_path = 'cifar-100-python/'
train_data_100 = unpickle(data_path + 'train')[b'data']
train_data_100 = train_data_100.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data_100 = unpickle(data_path + 'test')[b'data']
test_data_100 = test_data_100.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels_100 = np.array(unpickle(data_path + 'train')[b'fine_labels'])
test_labels_100 = np.array(unpickle(data_path + 'test')[b'fine_labels'])

In [5]:
# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Print the number of features, classes, and training samples for CIFAR-100
print("\nCIFAR-100")
print("Number of features:", train_data_100.shape[1:])
print("Number of classes:", len(np.unique(train_labels_100)))
print("Number of training samples:", train_data_100.shape[0])

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000

CIFAR-100
Number of features: (32, 32, 3)
Number of classes: 100
Number of training samples: 50000


In [6]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.5,1.5],
                             brightness_range=[0.5,1.0],
                             shear_range=0.5,
                             width_shift_range=0.5, 
                             height_shift_range=0.5,
                             vertical_flip=True,
                             horizontal_flip=True)

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [69]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import ReduceLROnPlateau

model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform', 
                   kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,l2=0.1)))
model_10.add(Dense(32, activation='relu', kernel_initializer='he_uniform', 
                   kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,l2=0.1)))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.01, momentum=0.9)

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_QAT.h5', save_freq='epoch')

lr_scheduler = ReduceLROnPlateau(monitor="val_accuracy",
                                factor=0.1,
                                patience=10,
                                verbose=0,
                                mode="max",
                                min_delta=0.0001,
                                cooldown=0,
                                min_lr=0.001)

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=50,validation_data=(x_test, y_test),callbacks=[checkpoint_callback,lr_scheduler])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_17 (Quantize  (None, 32, 32, 3)        3         
 Layer)                                                          
                                                                 
 quant_conv2d_159 (QuantizeW  (None, 32, 32, 32)       963       
 rapperV2)                                                       
                                                                 
 quant_conv2d_160 (QuantizeW  (None, 32, 32, 32)       9315      
 rapperV2)                                                       
                                                                 
 quant_max_pooling2d_80 (Qua  (None, 8, 8, 32)         1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_conv2d_161 (QuantizeW  (None, 8, 8, 64)       

KeyboardInterrupt: 

In [70]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

Quantization aware training loss:  1.1021994352340698
Quantization aware training accuracy:  0.7612000107765198


In [71]:
# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7yvskgue/assets


INFO:tensorflow:Assets written to: /tmp/tmp7yvskgue/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-15 01:29:42.391262: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-15 01:29:42.391293: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-15 01:29:42.391445: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp7yvskgue
2023-05-15 01:29:42.397527: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-15 01:29:42.397544: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp7yvskgue
2023-05-15 01:29:42.416555: I tensorf

In [72]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant_int8_qat)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)
# Save the quantized model to disk
open("cifar10_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


319088

In [73]:
tflite_interpreter = tf.lite.Interpreter('cifar10_qat_int8.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_conv2d_159_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>


In [74]:
predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()

In [75]:
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Accuracy of quantized to int8 model is 76.14%


In [76]:
# Strucutred pruning with constant sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=2000, frequency=100),
        'block_size': (1, 1),
}

# Create a pruning model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_model.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_QAT_St_Pruning.h5', save_freq='epoch')

# Train and evaluate the pruned model
# if you get an error like train_function -> train_function
# just run the cell again
pruned_model.fit(x_train,y_train, batch_size=64,epochs=5,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,checkpoint_callback])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 159 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 160 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_max_poo  (None, 8, 8, 32)         1         
 ling2d_80 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 8, 8, 64)         36930     
 161 (PruneLowMagnitude)                                         
                                                     

In [77]:
# Unstrucutred pruning with constant sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=2000, frequency=100),
}

# Create a pruning model
pruned_model_unstructured = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model_unstructured.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_model_unstructured.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_QAT_US_Pruning_Static.h5', save_freq='epoch')

# Train and evaluate the pruned model
pruned_model_unstructured.fit(x_train,y_train, batch_size=64,epochs=5,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,checkpoint_callback])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 159 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 160 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_max_poo  (None, 8, 8, 32)         1         
 ling2d_80 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 8, 8, 64)         36930     
 161 (PruneLowMagnitude)                                         
                                                     

In [78]:
# Unstructured pruning with dynamic sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                    final_sparsity=0.80,
                                                                    begin_step=2000,
                                                                    end_step=4000,
                                                                    frequency=100)

}

# Create a pruning model
pruned_model_unstructured_dynamic = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model_unstructured_dynamic.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_model_unstructured_dynamic.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_QAT_US_Pruning.h5', save_freq='epoch')


pruned_model_unstructured_dynamic.fit(x_train,y_train, batch_size=64,epochs=5,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,checkpoint_callback])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 159 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 160 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_max_poo  (None, 8, 8, 32)         1         
 ling2d_80 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 8, 8, 64)         36930     
 161 (PruneLowMagnitude)                                         
                                                     

In [79]:
import os

pruned_model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

pruned_keras_file = 'keras_model_10_QAT_St_Pruning.h5'
tf.keras.models.save_model(pruned_model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file))

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_for_export)
pruned_tflite_model = converter.convert()

pruned_tflite_file = 'model_10_QAT_St_Pruning.tflite'

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file))

# Conversion to TF Lite
pruned_model_unstructured_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model_unstructured)

pruned_keras_file_unstructured = 'keras_model_10_QAT_US_Pruning_Static.h5'
tf.keras.models.save_model(pruned_model_unstructured_for_export, pruned_keras_file_unstructured, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file_unstructured))

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_unstructured_for_export)
pruned_tflite_model_unstructured = converter.convert()

# Save the model
pruned_tflite_file_unstructured = 'model_10_QAT_US_Pruning_Static.tflite'

with open(pruned_tflite_file_unstructured, 'wb') as f:
    f.write(pruned_tflite_model_unstructured)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file_unstructured))

# Conversion to Keras Lite
pruned_model_unstructured_dynamic_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model_unstructured_dynamic)

pruned_keras_file_unstructured_dynamic = 'keras_model_10_QAT_US_Pruning.h5'
tf.keras.models.save_model(pruned_model_unstructured_dynamic_for_export, pruned_keras_file_unstructured_dynamic, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file_unstructured_dynamic))

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_unstructured_dynamic_for_export)
pruned_tflite_model_unstructured_dynamic = converter.convert()

# Save the model
pruned_tflite_file_unstructured_dynamic = 'model_10_QAT_US_Pruning.tflite'

with open(pruned_tflite_file_unstructured_dynamic, 'wb') as f:
    f.write(pruned_tflite_model_unstructured_dynamic)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file_unstructured_dynamic))

Saved pruned Keras model to: /home/themandalorian/Git/ML-on-Microcontrollers/keras_model_10_QAT_St_Pruning.h5


INFO:tensorflow:Assets written to: /tmp/tmph4fbur2r/assets


INFO:tensorflow:Assets written to: /tmp/tmph4fbur2r/assets


Saved pruned TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_QAT_St_Pruning.tflite


2023-05-15 01:37:22.687814: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-15 01:37:22.687842: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-15 01:37:22.687985: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmph4fbur2r
2023-05-15 01:37:22.690051: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-15 01:37:22.690064: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmph4fbur2r
2023-05-15 01:37:22.695589: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-15 01:37:22.720800: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmph4fbur2r
2023-05-15 01:37:22.729515: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 41531 m

...

Saved pruned Keras model to: /home/themandalorian/Git/ML-on-Microcontrollers/keras_model_10_QAT_US_Pruning.h5


INFO:tensorflow:Assets written to: /tmp/tmp_c0ggshd/assets


INFO:tensorflow:Assets written to: /tmp/tmp_c0ggshd/assets


Saved pruned TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_QAT_US_Pruning.tflite


2023-05-15 01:37:25.316766: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-15 01:37:25.316793: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-15 01:37:25.316935: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_c0ggshd
2023-05-15 01:37:25.318940: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-15 01:37:25.318954: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp_c0ggshd
2023-05-15 01:37:25.324498: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-15 01:37:25.349832: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp_c0ggshd
2023-05-15 01:37:25.358861: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 41925 m

In [80]:
test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of pruned_tflite model is {}%".format(accuracy_score*100))

Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of pruned_tflite model is 75.08%


In [81]:
test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file_unstructured))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of pruned tflite unstructured model is {}%".format(accuracy_score*100))

Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of pruned tflite unstructured model is 75.08%


In [82]:
test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file_unstructured_dynamic))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of pruned tflite unstructured dynamic is {}%".format(accuracy_score*100))

Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of pruned tflite unstructured dynamic is 75.08%


In [83]:
quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(
              pruned_model_unstructured_for_export)

pruned_qat_model = tfmot.quantization.keras.quantize_apply(quant_aware_annotate_model,
                   tfmot.experimental.combine.Default8BitPrunePreserveQuantizeScheme())

pruned_qat_model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_qat_model.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_QAT_Pruning.h5', save_freq='epoch')

# train the model
pruned_qat_model.fit(x_train,y_train, batch_size=64,epochs=5,validation_data=(x_test, y_test),
                                      callbacks=[checkpoint_callback])

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_18 (Quantize  (None, 32, 32, 3)        3         
 Layer)                                                          
                                                                 
 quant_conv2d_159 (QuantizeW  (None, 32, 32, 32)       963       
 rapperV2)                                                       
                                                                 
 quant_conv2d_160 (QuantizeW  (None, 32, 32, 32)       9315      
 rapperV2)                                                       
                                                                 
 quant_max_pooling2d_80 (Qua  (None, 8, 8, 32)         1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_conv2d_161 (QuantizeW  (None, 8, 8, 64)       

In [84]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_qat_tflite_model = converter.convert()

# Save the model
pruned_qat_tflite_file = 'model_10_QAT_Pruning.tflite'

with open(pruned_qat_tflite_file, 'wb') as f:
    f.write(pruned_qat_tflite_model)

print('Saved pruned QAT TFLite model to:', os.path.abspath(pruned_qat_tflite_file))

INFO:tensorflow:Assets written to: /tmp/tmp0xjidud6/assets


INFO:tensorflow:Assets written to: /tmp/tmp0xjidud6/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Saved pruned QAT TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_QAT_Pruning.tflite


2023-05-15 01:40:57.625552: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-15 01:40:57.625580: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-15 01:40:57.625727: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0xjidud6
2023-05-15 01:40:57.632210: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-15 01:40:57.632233: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0xjidud6
2023-05-15 01:40:57.652742: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-15 01:40:57.761252: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp0xjidud6
2023-05-15 01:40:57.788706: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 162977 

In [85]:
test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_qat_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of quantized to int8 model is 76.29%
